In [1]:
import sys
# Tested module
sys.path.append("/home/simon/Documents/REPO/Exact_laws")


In [ ]:
from exact_laws.el_calc_mod.grids import load_grid, load_incgrid_from_grid, load_listgrid_from_incgrid, reorganise_quantities
import numpy as np

In [ ]:
list_prim = dataset_terms.grid.coords['listprim']
list_sec = dataset_terms.grid.coords['listsec']
nb_sec_by_dirr = dataset_terms.grid.coords['nb_sec_by_dirr']
N = incgrid.spatial_grid.N
c = incgrid.spatial_grid.c

In [ ]:
output = {}
list_flux = []
for k in dataset_terms.quantities:
    if k.startswith("flux"):
        output["div_" + k] = np.zeros((len(list_prim)))
        term = dataset_terms.quantities[k]
        list_flux.append(k)

for ind, vect_prim in enumerate(list_prim):
    point_sec = [[], [], []]
    for dirr in range(3):
        for i in range(-nb_sec_by_dirr, nb_sec_by_dirr + 1):
            if not i == 0:
                vect = list(vect_prim)
                vect[dirr] = (
                    (vect[dirr] + i)
                    - (N[dirr] * ((vect[dirr] + i) >= (N[dirr] / 2)))
                    + (N[dirr] * ((vect[dirr] + i) <= (-N[dirr] / 2)))
                )
                vect = tuple(vect)
                loc = -1
                try:
                    index = list_prim.index(vect)
                    loc = 0
                except:
                    index = list_sec.index(vect)
                    loc = 1
                point_sec[dirr].append((loc, index))
    for t in list_flux:
        div_point = 0
        if len(point_sec[0]) == 0:
            div_point += np.nan
        else:
            for dirr in range(3):
                values = []
                for i in len(point_sec[dirr]):
                    values.append(dataset_terms.quantities[t][point_sec[dirr][i][0]][point_sec[dirr][i][1]])
                div_point += cdiff(
                    tab = values,
                    length_case=c[0],
                    precision=nb_sec_by_dirr * 2,
                    period=False,
                    point=True,
                )
        output["div_" + t][ind] = div_point
